In [1]:
import pandas as pd

pd.describe_option()
pd.set_option('display.max_columns',None)

customers_df= pd.read_csv('dataset/customers.csv')
orders_df= pd.read_csv('dataset/orders.csv')
order_items_df= pd.read_csv('dataset/order_items.csv')
payments_df= pd.read_csv('dataset/payments.csv')
products_df= pd.read_csv('dataset/products.csv')

compute.use_bottleneck : bool
    Use the bottleneck library to accelerate if it is installed,
    the default is True
    Valid values: False,True
    [default: True] [currently: True]
compute.use_numba : bool
    Use the numba engine option for select operations if it is installed,
    the default is False
    Valid values: False,True
    [default: False] [currently: False]
compute.use_numexpr : bool
    Use the numexpr library to accelerate computation if it is installed,
    the default is True
    Valid values: False,True
    [default: True] [currently: True]
display.chop_threshold : float or None
    if set to a float value, all float values smaller than the given threshold
    will be displayed as exactly 0 by repr and friends.
    [default: None] [currently: None]
display.colheader_justify : 'left'/'right'
    Controls the justification of column headers. used by DataFrameFormatter.
    [default: right] [currently: right]
display.date_dayfirst : boolean
    When True, prints an

In [2]:
# 가격 정의 - 어떤 컬럼들을 합쳐서 새로운 파생 변수로 만들었는지
# 가격 변동 있는 데이터 있는지.

# order_items_df: price shipping_charges / product_id order_item_id 
# payments_df: payment_sequential payment_installments payment_value / order_id 

#! pirce + shipping_charges + (payment_sequential 별 payment_value)

# order_item_id 가 여러개니까 order_id에 따라 병합
order_items_df[['order_id', 'order_item_id', 'price', 'shipping_charges']]
order_items_grouped_order_id= order_items_df.groupby('order_id').agg({
    'price': 'sum'
    , 'shipping_charges': 'sum'
}).reset_index()

order_items_grouped_order_id.head(3)

# order_items_grouped_order_id.isna().sum() 결측치 존재 X

,order_id,price,shipping_charges
0,00010242fe8c5a6d1ba2dd792cb16214,58.9,13.29
1,00018f77f2f0320c557190d7a144bdd3,239.9,19.93
2,000229ec398224ef6ca0657da4fc703e,199.0,17.87


In [3]:
order_items_df['order_item_id'].value_counts()
order_items_df[order_items_df['order_item_id']==21]

,order_id,order_item_id,product_id,seller_id,price,shipping_charges
57317,8272b63d03f5f79c56e9e4120aec44ef,21,79ce45dbc2ea29b22b5a261bbb7b7ee7,2709af9587499e95e803a6498a5a56e9,7.8,6.57


In [4]:
print(order_items_grouped_order_id['price'].equals(order_items_df['price']))
print(order_items_grouped_order_id['shipping_charges'].equals(order_items_df['shipping_charges']))

order_items_df[order_items_df['order_id']=='8272b63d03f5f79c56e9e4120aec44ef'].price.sum()

False
False


31.799999999999997

In [5]:
order_items_grouped_order_id[order_items_grouped_order_id['order_id']=='8272b63d03f5f79c56e9e4120aec44ef']
# 각 order_id별 order_item 병합 확인

,order_id,price,shipping_charges
50137,8272b63d03f5f79c56e9e4120aec44ef,31.8,164.37


In [8]:
# (payment_sequential 별 payment_value) 
payments_df.sort_values(by='payment_sequential').head(5)

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
68490,db974c12004fe92ab344e63e8e4b995f,1,credit_card,7,104.56
68489,71db292c69a2bff7186b705046336c61,1,credit_card,3,43.68
68486,a3a334e8cd38b52f151585bfa1e22190,1,wallet,1,98.51
68485,eb270c4cbf3671961f3883e8b888571b,1,wallet,1,106.97


In [7]:
# test_df= pd.merge(order_items_df, order_items_grouped_order_id, on= 'order_id', how= 'left')
# test_df

# test1=  order_items_grouped_order_id['total_price']
# test1

KeyError: 'total_price'

In [9]:
order_items_df['order_id'].unique()

array(['00010242fe8c5a6d1ba2dd792cb16214',
       '00018f77f2f0320c557190d7a144bdd3',
       '000229ec398224ef6ca0657da4fc703e', ...,
       'fffce4705a9662cd70adb13d4a31832d',
       'fffe18544ffabc95dfada21779c9644f',
       'fffe41c64501cc87c801fd61db3f6244'], dtype=object)

In [10]:
order_items_df[order_items_df['order_id']== 'fffe18544ffabc95dfada21779c9644f']

,order_id,order_item_id,product_id,seller_id,price,shipping_charges
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,55.99,8.72


In [11]:
payments_df[payments_df['order_id']== 'fa65dad1b0e818e3ccc5cb0e39231352'].payment_value.sum()

457.99

In [12]:
# -- order_id로 그룹화
payments_grouped_order_id= payments_df.groupby('order_id').agg({
    'payment_sequential': 'sum'
    , 'payment_value': 'sum'
}).reset_index()

payments_grouped_order_id.sort_values(by='payment_sequential', ascending=False).head(3)
# 29+28+27+26+...+3+2+1 = 435

,order_id,payment_sequential,payment_value
97261,fa65dad1b0e818e3ccc5cb0e39231352,435,457.99
79611,ccf804e764ed5650cd8759557269dc13,351,62.68
15577,285c2e15bebd4ac83635ccc563dc71f4,253,40.85


In [17]:
# -- order_id로 그룹화
payments_grouped_order_id= payments_df.groupby('order_id').agg({
    'payment_sequential': 'sum',
    'payment_value': 'sum'
}).reset_index()

payments_grouped_order_id.sort_values(by='payment_sequential', ascending=False).head(3)
# 29+28+27+26+...+3+2+1 = 435

##### paymet_value_sum으로 이름 재정의

,order_id,payment_sequential,payment_value
97261,fa65dad1b0e818e3ccc5cb0e39231352,435,457.99
79611,ccf804e764ed5650cd8759557269dc13,351,62.68
15577,285c2e15bebd4ac83635ccc563dc71f4,253,40.85


In [20]:
payments_df.sort_values(by='payment_sequential').head(3)

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
68490,db974c12004fe92ab344e63e8e4b995f,1,credit_card,7,104.56
68489,71db292c69a2bff7186b705046336c61,1,credit_card,3,43.68


In [ ]:
payments_grouped_order_id.isna().sum()

order_id              0
payment_sequential    0
payment_value         0
dtype: int64

In [ ]:
payments_grouped_order_id[payments_grouped_order_id['order_id']=='fa65dad1b0e818e3ccc5cb0e39231352']

,order_id,payment_sequential,payment_value
97261,fa65dad1b0e818e3ccc5cb0e39231352,435,457.99


In [ ]:
order_items_grouped_order_id[order_items_grouped_order_id['order_id']=='fa65dad1b0e818e3ccc5cb0e39231352']

,order_id,price,shipping_charges
96502,fa65dad1b0e818e3ccc5cb0e39231352,392.55,65.44


In [ ]:
order_items_df['order_item_id'].value_counts()
order_items_df[order_items_df['order_item_id']==21]
order_items_df[order_items_df['order_item_id']==12]

,order_id,order_item_id,product_id,seller_id,price,shipping_charges
11943,1b15974a0141d54e36626dca3fdc731a,12,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,100.00,10.12
19217,2c2a19b5703863c908512d135aa6accc,12,03e1c946c0ddfc58724ff262aef08dff,98dac6635aee4995d501a3972e047414,20.70,16.11
24561,37ee401157a3a0b28c9c6d0ed8c3b24b,12,d34c07a2d817ac73f4caf8c574215fed,e7d5b006eb624f13074497221eb37807,29.99,7.39
25575,3a213fcdfe7d98be74ea0dc05a8b31ae,12,a62e25e09e05e6faf31d90c6ec1aa3d1,634964b17796e64304cadf1ad3050fb7,108.00,15.52
29268,428a2f660dc84138d969ccd69a0ab6d5,12,89b190a046022486c635022524a974a8,f326006815956455b2859abd58fe7e39,65.49,16.22
43766,637617b3ffe9e2f7a2411243829226d0,12,bf44071ef18f5c9ded039681c68b1996,1f50f920176fa81dab994f9023523100,79.90,24.02
50925,73c8ab38f07dc94389065f7eba4f297a,12,422879e10f46682990de24d770e7f83d,1f50f920176fa81dab994f9023523100,59.00,13.43
57308,8272b63d03f5f79c56e9e4120aec44ef,12,270516a3f41dc035aa87d220228f844c,2709af9587499e95e803a6498a5a56e9,1.20,7.89
68196,9bdc4d4c71aa1de4606060929dee888c,12,44a5d24dd383324a421569ca697b13c2,e7d5b006eb624f13074497221eb37807,29.99,7.78
69651,9ef13efd6949e4573a18964dd1bbe7f5,12,37eb69aca8718e843d897aa7b82f462d,0b36063d5818f81ccb94b54adfaebbf5,51.00,1.20


In [ ]:
order_items_df[order_items_df['order_id']=='c05d6a79e55da72ca780ce90364abed9']

,order_id,order_item_id,product_id,seller_id,price,shipping_charges
84658,c05d6a79e55da72ca780ce90364abed9,1,710b7c26b7a742f497bba45fab91a25f,98dac6635aee4995d501a3972e047414,21.15,5.54
84659,c05d6a79e55da72ca780ce90364abed9,2,a9d9db064d4afd4458eb3e139fe29167,98dac6635aee4995d501a3972e047414,14.49,9.24
84660,c05d6a79e55da72ca780ce90364abed9,3,a9d9db064d4afd4458eb3e139fe29167,98dac6635aee4995d501a3972e047414,14.49,9.24
84661,c05d6a79e55da72ca780ce90364abed9,4,710b7c26b7a742f497bba45fab91a25f,98dac6635aee4995d501a3972e047414,21.15,5.54
84662,c05d6a79e55da72ca780ce90364abed9,5,a9d9db064d4afd4458eb3e139fe29167,98dac6635aee4995d501a3972e047414,14.49,9.24
84663,c05d6a79e55da72ca780ce90364abed9,6,710b7c26b7a742f497bba45fab91a25f,98dac6635aee4995d501a3972e047414,21.15,5.54
84664,c05d6a79e55da72ca780ce90364abed9,7,a9d9db064d4afd4458eb3e139fe29167,98dac6635aee4995d501a3972e047414,14.49,9.24
84665,c05d6a79e55da72ca780ce90364abed9,8,710b7c26b7a742f497bba45fab91a25f,98dac6635aee4995d501a3972e047414,21.15,5.54
84666,c05d6a79e55da72ca780ce90364abed9,9,a9d9db064d4afd4458eb3e139fe29167,98dac6635aee4995d501a3972e047414,14.49,9.24
84667,c05d6a79e55da72ca780ce90364abed9,10,710b7c26b7a742f497bba45fab91a25f,98dac6635aee4995d501a3972e047414,21.15,5.54


In [ ]:
# 설명
order_items_grouped_order_id['total_price']=order_items_grouped_order_id['price']+order_items_grouped_order_id['shipping_charges']
order_items_grouped_order_id[order_items_grouped_order_id['order_id']=='fa65dad1b0e818e3ccc5cb0e39231352']

,order_id,price,shipping_charges,total,total_price
96502,fa65dad1b0e818e3ccc5cb0e39231352,392.55,65.44,457.99,457.99


# 각 order_id로 그룹화한 price + payment_value = total_value

In [ ]:
value_grouped_df= pd.merge(order_items_grouped_order_id, payments_grouped_order_id, on= 'order_id', how= 'left')
value_grouped_df

value_grouped_df= value_grouped_df.drop(labels= 'payment_sequential', axis= 1)

In [ ]:
value_grouped_df['total_value']= value_grouped_df['price']+value_grouped_df['payment_value']
value_grouped_df['total_value_shipping']= value_grouped_df['total_value']+value_grouped_df['shipping_charges']

value_grouped_df.head()

,order_id,price,shipping_charges,payment_value,total_value,total_value_shipping
0,00010242fe8c5a6d1ba2dd792cb16214,58.90,13.29,72.19,131.09,144.38
1,00018f77f2f0320c557190d7a144bdd3,239.90,19.93,259.83,499.73,519.66
2,000229ec398224ef6ca0657da4fc703e,199.00,17.87,216.87,415.87,433.74
3,00024acbcdf0a6daa1e931b038114c75,12.99,12.79,25.78,38.77,51.56
4,00042b26cf59d7ce69dfabb4e55b4fd9,199.90,18.14,218.04,417.94,436.08


In [ ]:
value_grouped_df.isna().sum()
# ....???

order_id                0
price                   0
shipping_charges        0
payment_value           1
total_value             1
total_value_shipping    1
dtype: int64

In [ ]:
value_grouped_df= value_grouped_df[value_grouped_df['payment_value'].notna()]
value_grouped_df= value_grouped_df[value_grouped_df['total_value'].notna()]
value_grouped_df= value_grouped_df[value_grouped_df['total_value_shipping'].notna()]

value_grouped_df.isna().sum()

order_id                0
price                   0
shipping_charges        0
payment_value           0
total_value             0
total_value_shipping    0
dtype: int64

In [ ]:
value_grouped_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98665 entries, 0 to 98665
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   order_id              98665 non-null  object 
 1   price                 98665 non-null  float64
 2   shipping_charges      98665 non-null  float64
 3   payment_value         98665 non-null  float64
 4   total_value           98665 non-null  float64
 5   total_value_shipping  98665 non-null  float64
dtypes: float64(5), object(1)
memory usage: 5.3+ MB


# 가격 변동 있는 데이터 있는지.

In [ ]:
# order_items_df: product_id price 
order_items_df_price= order_items_df.groupby('product_id').agg({
    'price': 'std'
}).reset_index()

order_items_df_price['price'].value_counts()

price
0.000000     8934
7.071068      145
3.535534       59
14.142136      49
0.707107       39
             ... 
12.879510       1
2.640502        1
0.487463        1
6.032856        1
0.070711        1
Name: count, Length: 4412, dtype: int64